<a href="https://colab.research.google.com/github/Thiru08V/CS19P18_DLC-LAB_210701290/blob/main/EX_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BUILD GENERATIVE ADVERSARIAL NEURAL NETWORK

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

In [2]:
# Step 1: Load the MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Step 2: Preprocessing the dataset
# Normalize the images to the range [-1, 1]
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension for Conv2D

In [4]:
# Set constants
latent_dim = 100  # Size of the latent space (input to the generator)
img_shape = (28, 28, 1)  # Shape of the generated images

In [5]:
# Step 3: Build the generator
def build_generator():
    model = models.Sequential()

    # Fully connected layer
    model.add(layers.Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(layers.Reshape((7, 7, 128)))

    # Upsampling
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.ReLU())

    # Upsampling
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.ReLU())

    # Output layer
    model.add(layers.Conv2D(1, (3, 3), padding='same', activation='tanh'))

    return model

In [6]:
# Step 4: Build the discriminator
def build_discriminator():
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), input_shape=img_shape, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))

    # Flatten and output
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [7]:
# Step 5: Compile the GAN model
# Create the generator
generator = build_generator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Create the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [9]:
# The GAN model combines both the generator and discriminator
discriminator.trainable = False  # Freeze the discriminator during GAN training

In [10]:
# GAN Input
gan_input = layers.Input(shape=(latent_dim,))
generated_img = generator(gan_input)
validity = discriminator(generated_img)

In [11]:
# Create the GAN model (Generator -> Discriminator)
gan = models.Model(gan_input, validity)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [12]:
# Step 6: Training the GAN
def train_gan(epochs, batch_size=128, save_interval=100):
    half_batch = batch_size // 2

    for epoch in range(epochs):

        # Train the discriminator
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[idx]

        # Generate a half batch of fake images
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        # Labels for real and fake images
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Train the discriminator (real images = 1, fake images = 0)
        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        # Generate fake images with noise and label them as real (1)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, misleading_labels)

        # Save the generated images at save intervals
        if epoch % save_interval == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]:.4f}, D Acc: {100 * d_loss[1]:.2f}%, G Loss: {g_loss:.4f}")
            save_generated_images(epoch)

In [13]:
# Function to save generated images
def save_generated_images(epoch, num_images=10):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_imgs = generator.predict(noise)

    # Rescale images to [0, 1] range
    generated_imgs = 0.5 * generated_imgs + 0.5

    fig, axs = plt.subplots(1, num_images, figsize=(20, 2))
    for i in range(num_images):
        axs[i].imshow(generated_imgs[i, :, :, 0], cmap='gray')
        axs[i].axis('off')

    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.close()

In [16]:
# Step 7: Train the GAN for 10000 epochs
def train_gan(epochs=10000, batch_size=128, save_interval=1000):
    half_batch = batch_size // 2

    for epoch in range(epochs):

        # Train the discriminator
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[idx]

        # Generate a half batch of fake images
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        # Labels for real and fake images
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Train the discriminator (real images = 1, fake images = 0)
        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # Average of real and fake losses

        # Train the generator
        # Generate fake images with noise and label them as real (1)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, misleading_labels)

        # Save generated images at intervals
        if epoch % save_interval == 0:
            d_loss_value = d_loss[0]
            d_acc = d_loss[1]
            g_loss_value = g_loss
            print(f"Epoch {epoch}, D Loss: {d_loss_value:.4f}, D Acc: {100 * d_acc:.2f}%, G Loss: {g_loss_value:.4f}")
            save_generated_images(epoch)

In [17]:
# Function to save generated images
def save_generated_images(epoch, num_images=10):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_imgs = generator.predict(noise)

    # Rescale images to [0, 1] range
    generated_imgs = 0.5 * generated_imgs + 0.5

    fig, axs = plt.subplots(1, num_images, figsize=(20, 2))
    for i in range(num_images):
        axs[i].imshow(generated_imgs[i, :, :, 0], cmap='gray')
        axs[i].axis('off')

    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.close()

In [23]:
# Step 7: Train the GAN for 10000 epochs
def train_gan(epochs=10000, batch_size=128, save_interval=1000):
    half_batch = batch_size // 2

    for epoch in range(epochs):

        # Train the discriminator
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[idx]

        # Generate a half batch of fake images
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        # Labels for real and fake images
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Train the discriminator (real images = 1, fake images = 0)
        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  # Average of real and fake losses

        # Train the generator
        # Generate fake images with noise and label them as real (1)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, misleading_labels)

        # Save generated images at intervals
        if epoch % save_interval == 0:
            d_loss_value = d_loss[0]  # Assuming d_loss is a list and the first element is the loss
            d_acc = d_loss[1]      # Assuming the second element is accuracy

            # Handle the g_loss value, which is a list
            if isinstance(g_loss, list):
                g_loss_value = g_loss[0]  # Take the first value from the list
            else:
                g_loss_value = g_loss  # If it's not a list, use it directly

            # Check if d_loss_value, d_acc, g_loss_value are numeric
            if not all(isinstance(x, (int, float)) for x in [d_loss_value, d_acc, g_loss_value]):
                print("Warning: Loss or accuracy values are not numeric. Check the output of train_on_batch.")
                print("d_loss_value:", d_loss_value, type(d_loss_value))
                print("d_acc:", d_acc, type(d_acc))
                print("g_loss_value:", g_loss_value, type(g_loss_value))

            print(f"Epoch {epoch}, D Loss: {d_loss_value:.4f}, D Acc: {100 * d_acc:.2f}%, G Loss: {g_loss_value:.4f}")

# Call train_gan to start training
train_gan(epochs=100, batch_size=128, save_interval=1000)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
d_loss_value: 1.0993795 <class 'numpy.float32'>
d_acc: 0.3328289 <class 'numpy.float32'>
g_loss_value: 1.0995939 <class 'numpy.ndarray'>
Epoch 0, D Loss: 1.0994, D Acc: 33.28%, G Loss: 1.0996
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
2/2 ━━━━━━